# Mapping migration

Introduction to vector data operations

## STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [11]:
%store -r pelican_occurrence_df pelican_eco_gdf

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>In the imports cell, we’ve included some packages that you will need.
Add imports for packages that will help you:</p>
<ol type="1">
<li>Make interactive maps with vector data</li>
<li>Access pre-defined month names</li>
<li>Define Coordinate Reference Systems (CRSs)</li>
</ol></div></div>

In [6]:
# Get month names
import calendar

# Libraries for Dynamic mapping
import cartopy.crs as ccrs
import panel as pn
import hvplot.pandas

### Create a simplified `GeoDataFrame` for plotting

Plotting larger files can be time consuming. The code below will
streamline plotting with `hvplot` by simplifying the geometry,
projecting it to a Mercator projection that is compatible with
`geoviews`, and cropping off areas in the Arctic.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Simplify ecoregion data</div></div><div class="callout-body-container callout-body"><p>Download and save ecoregion boundaries from the EPA:</p>
<ol type="1">
<li>Simplify the ecoregions with <code>.simplify(.05)</code>, and save
it back to the <code>geometry</code> column.</li>
<li>Change the Coordinate Reference System (CRS) to Mercator with
<code>.to_crs(ccrs.Mercator())</code></li>
<li>Use the plotting code that is already in the cell to check that the
plotting runs quickly (less than a minute) and looks the way you want,
making sure to change <code>gdf</code> to YOUR <code>GeoDataFrame</code>
name.</li>
</ol></div></div>

In [7]:
# Simplify the geometry to speed up processing
pelican_eco_gdf.geometry = pelican_eco_gdf.simplify(.01, preserve_topology=False)

# Change the CRS to Mercator for mapping
pelican_eco_gdf = pelican_eco_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
# This did take 1 minute and 13 seconds on Maddy's machine
pelican_eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())

:Polygons   [Longitude,Latitude]

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Map migration over time</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>If applicable, replace any variable names with the names you defined
previously.</li>
<li>Replace <code>column_name_used_for_ecoregion_color</code> and
<code>column_name_used_for_slider</code> with the column names you wish
to use.</li>
<li>Customize your plot with your choice of title, tile source, color
map, and size.</li>
</ol>
<div data-__quarto_custom="true" data-__quarto_custom_type="Callout"
data-__quarto_custom_context="Block" data-__quarto_custom_id="3">
<div data-__quarto_custom_scaffold="true">

</div>
<div data-__quarto_custom_scaffold="true">
<p>Your plot will probably still change months very slowly in your
Jupyter notebook, because it calculates each month’s plot as needed.
Open up the saved HTML file to see faster performance!</p>
</div>
</div></div></div>

In [14]:
pelican_eco_gdf = pelican_eco_gdf.rename_axis('ecoregion')

In [17]:
pelican_occurrence_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [21]:
# Join the occurrences with the plotting GeoDataFrame
pelican_occurrence_gdf = pelican_eco_gdf.join(pelican_occurrence_df[['norm_occurrences']])

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = pelican_occurrence_gdf.total_bounds

# Plot occurrence by ecoregion and month
pelican_migration_plot = (
    pelican_occurrence_gdf
    .hvplot.polygons(
        c='norm_occurrences',
        groupby='month',
        # Get the slider set up
        widget_location='bottom',
        # Use background tiles
        geo=True, 
        crs=ccrs.PlateCarree(), 
        tiles='CartoLight',
        project=True,
        color='norm_occurrences',
        title="American White Pelican Migration",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600
    )
)

# Save the plot
pelican_migration_plot.save('pelican_migration.html', embed=True)

pelican_migration_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'ef9dc0d3-0d9b-4282-93ed-50b34ca10d18': {'version…

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?: Fix the month labels</div></div><div class="callout-body-container callout-body"><p>Notice that the <code>month</code> slider displays numbers instead of
the month name. Use <code>pn.widgets.DiscreteSlider()</code> with the
<code>options=</code> parameter set to give the months names. You might
want to try asking ChatGPT how to do this, or look at the documentation
for <code>pn.widgets.DiscreteSlider()</code>. This is pretty tricky!</p></div></div>

In [19]:
discrete_slider = pn.widgets.DiscreteSlider(
    name = 'months',
    options = {1: 'January', 
           2: 'February', 
           3: 'March', 
           4: 'April', 
           5: 'May', 
           6: 'June', 
           7: 'July', 
           8: 'August', 
           9: 'September',
           10: 'October',
           11: 'November',
           12: 'December'
        }   
)



